In [1]:
import numpy
import pandas as pd
import glob
import time
import numpy as np
import math
from functools import reduce
import scipy.ndimage.filters as ndif
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
from scipy import stats
import random

from openpyxl import load_workbook
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Layout
from IPython.display import display
error = widgets.Valid(
    value=False,
    #description='Valid: ',
)

In [2]:
def residual(rho_S, rho_P, M1, M2, MS_d, MP_d, name_s, group_s, vk, Rk, Qk, name_p, group_p, vk_p, Rk_p, Qk_p, a, w1, DP, T, printed):

    # SPECIES 1 = SOLVENT
    # SPECIES 2 = POLYMER

    #############################
    # CONVERT to volume fraction

    vol_frac1 = (w1/rho_S)/((w1/rho_S)+((1-w1)/rho_P))
    vol_frac2 = 1 - vol_frac1
    w2 = 1 - w1 # Weight fraction

    ############################
    # CONVERT to mol fraction
    # take 1 kg basis
    m1 = 1000*w1 # mass of species 1 (in grams)
    m2 = 1000*w2 # mass of species 2 (in grams)

    n1 = m1/M1 # moles of species 1
    n2 = m2/M2 # moles of species 2

    x1 = n1/(n1+n2) # mol fraction of species 1
    x2 = 1 - x1 # mol fraction of species 2

    ################
    # Populate Dataframe with data
    # All the names are unique anyways, so this shouldn't cause an issue

    dict_ks = dict(zip(name_s, group_s))
    dict_kp = dict(zip(name_p, group_p))

    dict_Rks = dict(zip(name_s, Rk))
    dict_Rkp = dict(zip(name_p, Rk_p))

    dict_Qks = dict(zip(name_s, Qk))
    dict_Qkp = dict(zip(name_p, Qk_p))

    dict_Vks = dict(zip(name_s, vk))
    dict_Vkp = dict(zip(name_p, vk_p))

    index_array = np.unique(np.append(name_p, name_s))

    rows = []
    for n in range(0, len(index_array)):
        try:
            k_val = dict_ks[index_array[n]]
        except:
            k_val = dict_kp[index_array[n]]
        try:
            Rk_val = dict_Rks[index_array[n]]
        except:
            Rk_val = dict_Rkp[index_array[n]]
        try:
            Qk_val = dict_Qks[index_array[n]]
        except:
            Qk_val = dict_Qkp[index_array[n]]
        try:
            Vk_val1 = dict_Vks[index_array[n]]
        except:
            Vk_val1 = 0
        try:
            Vk_val2 = dict_Vkp[index_array[n]]
        except:
            Vk_val2 = 0
        row = [k_val, Rk_val, Qk_val, Vk_val1, Vk_val2]
        rows.append(row)

    df = pd.DataFrame(data=rows, index=index_array, columns=["k", "Rk", "Qk", "V_k^(1)", "V_k^(2)"])
    df = df.sort_values(by=['k'])
    if printed:
        #print(index_array)
        #print(rows)
        print(df)
        print()

    ##################
    # Calculate Interaction Parameter
    # Note parameters obtained automatically from parameter excel file (that is the "a" array below)

    psi = []
    for i in range(0, len(a)):
        psi_j = np.zeros(len(a[i]))
        for j in range(0, len(a[i])):
            psi_j[j] = np.exp(-a[i][j]/T)
        psi.append(psi_j)

    #print("Interaction Paramater")
    #print(psi)
    #print()
    ##############
    # Calculate R value (species based)

    r1_a = np.zeros(len(df.index))
    for i in range(0, len(df.index)):
        r1_a[i] = df["V_k^(1)"][i]*df["Rk"][i]
    r1 = np.sum(r1_a)

    # r_i' is the parameter related to molar group volume
    # R_k: molar group volume, obtained from normalizing
    # the Van Der Waals group volume by a CH2- group in polyethylene
    # ^ This R_k parameter is similarly obtained from a database

    #print("r1: " + str(r1))

    r2_a = np.zeros(len(df.index))
    for i in range(0, len(df.index)):
        r2_a[i] = df["V_k^(2)"][i]*df["Rk"][i]
    r2 = np.sum(r2_a)

    #print("r2: " + str(r2))
    #print()
    ##############
    # Calculate Q value (species based)

    q1_a = np.zeros(len(df.index))
    for i in range(0, len(df.index)):
        q1_a[i] = df["V_k^(1)"][i]*df["Qk"][i]
    q1 = np.sum(q1_a)

    # q_i' is the sum of group area parameter divided by molecular weight
    # Q_k is the group area parameter, obtained from normalizing
    # the Van der Waals surface area by a CH2
    # ^ This Q_k parameter is similarly obtained from a database

    #print("q1: " + str(q1))

    q2_a = np.zeros(len(df.index))
    for i in range(0, len(df.index)):
        q2_a[i] = df["V_k^(2)"][i]*df["Qk"][i]
    q2 = np.sum(q2_a)

    #print("q2: " + str(q2))
    #print()
    ##############
    # Calculate e_ki value (k = group, i = species)

    e_ki = []
    for k in range(0, len(df.index)):
        e_i = np.zeros(2) # b/c 2 species
        e_i[0] =  df["V_k^(1)"][k]*df["Qk"][k]/q1
        e_i[1] =  df["V_k^(2)"][k]*df["Qk"][k]/q2

        e_ki.append(e_i)
    # Just a convenient grouping to simplify later calculations
    # For every subgroup, k, this calculates v_k*Q_k/qi for both species 1 and 2 (i = species)
    # recall, qi is molar group volume of the species,
    # So this is the: molar volume of the subgroup of each species
    # divided by the molar group volume of the species

    #print("e_ki")
    #print(e_ki)
    #print()
    ##############
    # Calculate B_ik (i = species, k = group)

    B_ik = []

    for i in range(0, 2): # b/c 2 species
        B_k = np.zeros(len(df.index))
        for k in range(0, len(df.index)):
            B = np.zeros(len(df.index))
            for m in range(0, len(df.index)):
                B[m] = e_ki[m][i]*psi[m][k]
            B_k[k] = np.sum(B)
        B_ik.append(B_k)

    # Just a convenient grouping to simplify later calculations
    # This just multiplies e_ki from before with its corresponding group interaction parameter
    # then it sums it up for each subgroup
    # if you look at the residual eqn. below it becomes a bit clearer
    # theta_k[k]*(B_ik[0][k]/S_k[k]) = sum_m (Theta_m' * Psi_km) / sum_m( sum_n Theta_n' psi_nm)
    # here S_k[k] = sum_m( sum_n Theta_n' psi_nm)
    # and: theta_k[k]*B_ik[0][k] = sum_m (Theta_m' * Psi_km)

    #print("B_ik")
    #print(B_ik)
    #print()
    ##############
    # Calculate theta_k (k = group)

    theta_k = np.zeros(len(df.index))
    for k in range(0, len(df.index)):
        # for 2 species
        top = x1*q1*e_ki[k][0] + x2*q2*e_ki[k][1]
        bottom = x1*q1 + x2*q2
        theta_k[k] = top/bottom

    # This is the area fraction of group m (note: k is subgroup)

    #print("theta_k")
    #print(theta_k)
    #print()
    ##############
    # Calculate S_k (k = group)

    S_k = np.zeros(len(df.index))
    for k in range(0, len(df.index)):
        S = np.zeros(len(df.index))
        for m in range(0, len(df.index)):
            S[m] = theta_k[m]*psi[m][k]
        S_k[k] = np.sum(S)

    # This is just a convenient grouping, to simplify calculations
    # S_k = the sum over the subgroups of the product of theta_k and psi_[m][k]
    # it is used as the denominator in the calculation of the residual

    #print("S_k")
    #print(S_k)
    #print()

    ##################
    # Calculate residual activity COEFFICIENT (not activity like before)

    # Solvent Activity Coeffiecient Residual part

    part = np.zeros(len(df.index))
    for k in range(0, len(df.index)):
        part[k] = theta_k[k]*(B_ik[0][k]/S_k[k]) - e_ki[k][0]*np.log(B_ik[0][k]/S_k[k])

    second_part = np.sum(part)

    lnY1_R = q1*(1 - second_part)

    #print("lnY1_R")
    #print(lnY1_R)
    #print()

    return lnY1_R

def UNIFAC_FV(rho_S, rho_P, M1, M2, MS_d, MP_d, name_s, group_s, vk, Rk, Qk, name_p, group_p, vk_p, Rk_p, Qk_p, a, frac, DP, T, frac_type, printed):

    # SPECIES 1 = SOLVENT
    # SPECIES 2 = POLYMER

    # Sometimes its easier to define by vol_frac rather than by weight fraction, it depends on who you are comparing to
    if frac_type == 1:
        # Solvent Weight Fraction
        w1 = frac
        w2 = 1 - w1 # Weight fraction of species 2 (Polymer)
        vol_frac2 = rho_S*(1 - w1)/(w1*rho_P + rho_S*(1-w1))
        vol_frac1 = 1 - vol_frac2
    elif frac_type == 2:
        # Solvent Volume Fraction
        vol_frac1 = frac
        vol_frac2 = 1 - vol_frac1
        w1 = ((1-vol_frac2)*rho_S)/( vol_frac2*rho_P + (1 - vol_frac2)*rho_S)
        w2 = 1 - w1 # Weight fraction of species 2 (Polymer)
    elif frac_type == 3:
        # Polymer Weight Fraction
        w2 = frac
        w1 = 1 - w2 # Weight fraction of species 1 (Solvent)
        vol_frac2 = rho_S*(1 - w1)/(w1*rho_P + rho_S*(1-w1))
        vol_frac1 = 1 - vol_frac2
    elif frac_type == 4:
        # Polymer Volume Fraction
        vol_frac2 = frac
        vol_frac1 = 1 - vol_frac2
        w1 = ((1-vol_frac2)*rho_S)/( vol_frac2*rho_P + (1 - vol_frac2)*rho_S)
        w2 = 1 - w1 # Weight fraction of species 2 (Polymer)
    else:
        display(error)
        print("This error should never happen.")
        print("It indicates that a fraction type was not selected, but there is a default case.")
        print("If you see this error something has gone wrong.")
        return
    #######################################################
    # RESIDUAL SECTION

    lnY1_R = residual(rho_S, rho_P, M1, M2, MS_d, MP_d, name_s, group_s, vk, Rk, Qk, name_p, group_p, vk_p, Rk_p, Qk_p, a, w1, DP, T, printed)
    ###########################

    #print("vol_frac solvent(1): " + str(vol_frac1))
    #print("vol_frac polymer(2): " + str(vol_frac2))

    #print("Rho_1: " + str(rho_S))
    #print("Rho_2: " + str(rho_P))
    #######################################################
    # COMBINATORIAL SECTION
    # SOLVENT
    vk_Rk_sum = 0
    vk_Qk_sum = 0
    for i in range(0, len(vk)):
        vk_Rk_sum += vk[i]*Rk[i]
        vk_Qk_sum += vk[i]*Qk[i]

    r1p = vk_Rk_sum/M1 # r1p = r_1 prime (solvent)
    q1p = vk_Qk_sum/M1

    # POLYMER
    vk_Rk_sum_p = 0
    vk_Qk_sum_p = 0
    for i in range(0, len(vk_p)):
        vk_Rk_sum_p += vk_p[i]*Rk_p[i]
        vk_Qk_sum_p += vk_p[i]*Qk_p[i]

    r2p = vk_Rk_sum_p/M2 # r2p = r_2 prime (polymer)
    q2p = vk_Qk_sum_p/M2

    phi_1p = r1p*w1/(r1p*w1 + r2p*w2)   # phi 1 prime
    phi_2p = 1 - phi_1p

    theta_1p = q1p*w1/(q1p*w1 + q2p*w2)
    theta_2p = 1 - theta_1p
    #print(phi_1p)
    #print(theta_1p)

    z = 5 # z is coordination number = 10, but later divide by 2, so 5

    # combinatorial part
    ln_a1C = np.log(phi_1p) + phi_2p + z*M1*q1p*np.log(theta_1p/phi_1p) - z*M1*q1p*(1 - (phi_1p/theta_1p))
    # TEST (taking into account m )
    #ln_a1C = np.log(phi_1p) + phi_2p*(1 - 1/DP) + z*M1*q1p*np.log(theta_1p/phi_1p) - z*M1*q1p*(1 - (phi_1p/theta_1p))

    #######################################################

    # FREE VOLUME
    b = 1.28 # proportional factor of order unity, (empirical) [Oishi + Prausnitz]

    # More accurate to use a b value that is temeprature dependent like:
    # b = 2.98 - 0.00508*T <-- Specific to one system, fitted, don't use
    # https://sci-hub.se/https://doi.org/10.1007/BF00649090
    # "Figures 1 and 2 show detailed results for the benzenepolyethylene oxide system at 373K
    # and for the n-hexane - n-hexadecane system at 313 K. In both cases the activity
    # predicted by UNIFAC-FV without a temperature-dependent term is too low. A similar
    # tendency was observed in all other cases. "
    # "Values for A and B of 2.98 and -5.08x10^-3 were obtained using the
    # Marquardt optimization method."
    # Note "Marquardt optimization method" is just a fitting method, they are essentially
    # just adding a fitting parameter to match the activities.
    # IDEALLY: Try to find a method that calculates b using a group contribution method
    # rather than just parameterization. <-- Couldn't find, it works well enough how it is

    v_1b = (1/rho_S)/(15.17*b*r1p)

    v_M = ( (w1/rho_S) + (w2/rho_P) )/(15.17*b*(r1p*w1 + r2p*w2))

    c_1 = 1.1 # number of external degrees of freedom per solvent molecule
    # not sure how you calculate it but 1978 Oishi used 1.1
    # Some use different values of this ex:
    # Varying C: https://pubs.acs.org/doi/pdf/10.1021/i200018a033
    # Not calculated, its empirical parameter, i.e. fitted to data, and that just happened
    # to work best for the ~40 or so systems Oishi and Prausnitz attempted

    ln_a1FV = 3*c_1*np.log( (v_1b**(1/3) - 1)/(v_M**(1/3) - 1) ) - c_1*( ( (v_1b/v_M) -1 )*(1 - ( 1/(v_1b**(1/3)) ) )**(-1) )

    ####################
    b = 1
    # https://reader.elsevier.com/reader/sd/pii/S0378381204001888?token=D1C2B6F938D8FA0A47AB6EA102D59008E1773C1645A5AE56427A4B68BE75B93D4A233EEB541B0FA5A66EF8912C282DF9&originRegion=us-east-1&originCreation=20210928170854
    # Vapor–liquid equilibria in dendrimer and hyperbranched polymer solutions: experimental
    # data and modeling using UNIFAC-FV Matthias Seiler a, Jörn Rolker a, LyudmilaV.
    # Mokrushina b, Holger Kautz c, Holger Frey c, Wolfgang Arlt a,∗
    # They used b = 1, for the same reason I do:
    # As discussed by Lindvig et al. [23] and Elbro et al. [24], the recommended b-value
    # of 1.28 (see Eq. (A.10) in Appendix A.1) is not applicable to macromolecules where
    # the hard core volume is larger than the real molar polymer volume. Consequently, for
    # the investigated hyperbranched polymer systems, the b-parameter of 1.0 was adopted.
    # This is an old problem they cite a 1989 paper that also used this value of b parameter
    # for this reason

    v_1b_b1 = (1/rho_S)/(15.17*b*r1p)

    v_M_b1 = ( (w1/rho_S) + (w2/rho_P) )/(15.17*b*(r1p*w1 + r2p*w2))

    c_1 = 1.1 # number of external degrees of freedom per solvent molecule
    # not sure how you calculate it but 1978 Oishi used 1.1

    ln_a1FV_b1 = 3*c_1*np.log( (v_1b_b1**(1/3) - 1)/(v_M_b1**(1/3) - 1) ) - c_1*( ( (v_1b_b1/v_M_b1) -1 )*(1 - ( 1/(v_1b_b1**(1/3)) ) )**(-1) )

    ###################
    # In the same paper that had b=1 they mention that this is only needed when the hardcore
    # volme is larger than the real molar volume, so you can make the case that only when
    # this occurs that we set b to 1
    b = 1.28
    v_1b_b = (1/rho_S)/(15.17*b*r1p)
    if (v_1b_b < 1):
        b = 1
        v_1b_b = (1/rho_S)/(15.17*b*r1p)
    b = 1.28
    v_M_b = ( (w1/rho_S) + (w2/rho_P) )/(15.17*b*(r1p*w1 + r2p*w2))
    if (v_M_b < 1):
        b = 1
        v_M_b = ( (w1/rho_S) + (w2/rho_P) )/(15.17*b*(r1p*w1 + r2p*w2))

    c_1 = 1.1 # number of external degrees of freedom per solvent molecule

    ln_a1FV_b = 3*c_1*np.log( (v_1b**(1/3) - 1)/(v_M**(1/3) - 1) ) - c_1*( ( (v_1b/v_M) -1 )*(1 - ( 1/(v_1b**(1/3)) ) )**(-1) )


    #print("ln_a1FV: " + str(ln_a1FV))
    #print("ln_a1FV (b=1): " + str(ln_a1FV_b1))
    #print("ln_a1FV (b=1 or 1.28): " + str(ln_a1FV_b))
    #print()
    #######################################################

    ln_a1 = ln_a1C + ln_a1FV
    a1 = np.exp(ln_a1)*np.exp(lnY1_R)

    ln_a1_b1 = ln_a1C + ln_a1FV_b1
    a1_b1 = np.exp(ln_a1_b1)*np.exp(lnY1_R)

    ln_a1_b = ln_a1C + ln_a1FV_b
    a1_b = np.exp(ln_a1_b)*np.exp(lnY1_R)

    return a1, vol_frac2, a1_b1, a1_b

# Note: It is important to compare apples to apples
# Don't fit using ln(a) in python and compare to a fitting done using a in excel
# You will get different answers
def func(vol_frac2, X, DP):
    lna = np.log(1 - vol_frac2) + (1 - 1/DP)*vol_frac2 + X*vol_frac2**2
    return np.exp(lna)

def flory(vol_frac2, a, DP):
    xData = vol_frac2
    yData = a
    popt, pcov = curve_fit(func, xData, yData, bounds=([0, DP-0.0001], [10, DP+0.00001]))
    # DP here shouldn't vary but the scipy implementation has variation, so we add
    # constraints on the DP (+ or - 0.00001), the accuracy of UNIFAC-FV is only 2
    # significant figures anyways so this won't effect the overall accuracy
    print("_________________________________________________")
    print("Flory-Huggins Parameter: " + str(popt[0]))
    print("_________________________________________________")
    return popt[0]



###############################################################################
"""
=================================
Molar Mass Calculator
Author: Elijah Lopez
Version: 1.2.1
Last Updated: April 4th 2020
Created: July 8th 2017
Python Version: 3.6+
=================================
I modified this script a bit
"""

MM_of_Elements = {'H': 1.00794, 'He': 4.002602, 'Li': 6.941, 'Be': 9.012182, 'B': 10.811, 'C': 12.0107, 'N': 14.0067,
                  'O': 15.9994, 'F': 18.9984032, 'Ne': 20.1797, 'Na': 22.98976928, 'Mg': 24.305, 'Al': 26.9815386,
                  'Si': 28.0855, 'P': 30.973762, 'S': 32.065, 'Cl': 35.453, 'Ar': 39.948, 'K': 39.0983, 'Ca': 40.078,
                  'Sc': 44.955912, 'Ti': 47.867, 'V': 50.9415, 'Cr': 51.9961, 'Mn': 54.938045,
                  'Fe': 55.845, 'Co': 58.933195, 'Ni': 58.6934, 'Cu': 63.546, 'Zn': 65.409, 'Ga': 69.723, 'Ge': 72.64,
                  'As': 74.9216, 'Se': 78.96, 'Br': 79.904, 'Kr': 83.798, 'Rb': 85.4678, 'Sr': 87.62, 'Y': 88.90585,
                  'Zr': 91.224, 'Nb': 92.90638, 'Mo': 95.94, 'Tc': 98.9063, 'Ru': 101.07, 'Rh': 102.9055, 'Pd': 106.42,
                  'Ag': 107.8682, 'Cd': 112.411, 'In': 114.818, 'Sn': 118.71, 'Sb': 121.760, 'Te': 127.6,
                  'I': 126.90447, 'Xe': 131.293, 'Cs': 132.9054519, 'Ba': 137.327, 'La': 138.90547, 'Ce': 140.116,
                  'Pr': 140.90465, 'Nd': 144.242, 'Pm': 146.9151, 'Sm': 150.36, 'Eu': 151.964, 'Gd': 157.25,
                  'Tb': 158.92535, 'Dy': 162.5, 'Ho': 164.93032, 'Er': 167.259, 'Tm': 168.93421, 'Yb': 173.04,
                  'Lu': 174.967, 'Hf': 178.49, 'Ta': 180.9479, 'W': 183.84, 'Re': 186.207, 'Os': 190.23, 'Ir': 192.217,
                  'Pt': 195.084, 'Au': 196.966569, 'Hg': 200.59, 'Tl': 204.3833, 'Pb': 207.2, 'Bi': 208.9804,
                  'Po': 208.9824, 'At': 209.9871, 'Rn': 222.0176, 'Fr': 223.0197, 'Ra': 226.0254, 'Ac': 227.0278,
                  'Th': 232.03806, 'Pa': 231.03588, 'U': 238.02891, 'Np': 237.0482, 'Pu': 244.0642, 'Am': 243.0614,
                  'Cm': 247.0703, 'Bk': 247.0703, 'Cf': 251.0796, 'Es': 252.0829, 'Fm': 257.0951, 'Md': 258.0951,
                  'No': 259.1009, 'Lr': 262, 'Rf': 267, 'Db': 268, 'Sg': 271, 'Bh': 270, 'Hs': 269, 'Mt': 278,
                  'Ds': 281, 'Rg': 281, 'Cn': 285, 'Nh': 284, 'Fl': 289, 'Mc': 289, 'Lv': 292, 'Ts': 294, 'Og': 294,
                  '': 0, 'A': 0, 'M': 54.938045}
# This A at the end is in UNIFAC, ex: in 'ACH' A is not an actual element so set to 0

# Not sure why they mixed chemical formulas with non-chemical (shorthand) names for
# chemicals http://www.ddbst.com/published-parameters-unifac.html
# Things I changed in parameter file ^^^
# Changed CL to Cl, SI to Si, and BR to Br (so it can be read correctly)
# SU to S (Sulfur?)

# NOT sure about some of these like:
# Name: What I think it is
# ACRY: Acryilic acid
# ACF: Actinium(III) fluoride
# DMF: Dimethylformamide
# NMP: N-Methylpyrrolidone
# AMH2: hydroxymanganese  ???
# I'm just going to treat M as Mn (manganese)
# I'm not sure if it's right but they didn't put a name on it so I have no idea
# MORPH: morpholine
# BTI: (Bis(trifluoroacetoxy)iodo)benzene

exceptions = {'THF': 72.11, 'FURFURAL': 96.0846, 'DOH': 20.028, 'DMSO': 78.13, 'ACRY': 72.06,
             'ACF': 284.02, 'DMF': 73.09, 'NMP': 99.13, 'AMH2': 71.945, 'IMIDAZOL': 68.077,
             'MORPH': 87.12036, 'BTI': 430.041}

def molar_mass(compound: str, decimal_places=None) -> float:
    is_polyatomic = end = multiply = False
    polyatomic_mass, m_m, multiplier = 0, 0, 1
    element = ''

    if compound in exceptions:
        return exceptions[compound]
    else:
        for e in compound:
            if is_polyatomic:
                if end:
                    is_polyatomic = False
                    m_m += int(e) * polyatomic_mass if e.isdigit() else polyatomic_mass + MM_of_Elements[e]
                elif e.isdigit():
                    multiplier = int(str(multiplier) + e) if multiply else int(e)
                    multiply = True
                elif e.islower():
                    element += e
                elif e.isupper():
                    polyatomic_mass += multiplier * MM_of_Elements[element]
                    element, multiplier, multiply = e, 1, False
                elif e == ')':
                    polyatomic_mass += multiplier * MM_of_Elements[element]
                    element, multiplier = '', 1
                    end, multiply = True, False
            elif e == '(':
                m_m += multiplier * MM_of_Elements[element]
                element, multiplier = '', 1
                is_polyatomic, multiply = True, False
            elif e.isdigit():
                multiplier = int(str(multiplier) + e) if multiply else int(e)
                multiply = True
            elif e.islower():
                element += e
            elif e.isupper():
                m_m += multiplier * MM_of_Elements[element]
                element, multiplier, multiply = e, 1, False
        m_m += multiplier * MM_of_Elements[element]
        if decimal_places is not None:
            return round(m_m, decimal_places)
    return m_m

def input_UFV(name_S, name_P, vk_S, vk_P, rho_S, rho_P, DP, T, frac, frac_type, file_log):

    # Initial Check for errors

    # Check if T > 0
    if T <= 0:
        display(error)
        print("Please input a temperature greater than 0.")
        print("Note: The temperature unit is Kelvin.")
        file_log.write("Please input a temperature greater than 0.\n")
        file_log.write("Note: The temperature unit is Kelvin.\n")
        return -1, -1, -1

    # Check if densities non-negative
    if rho_S <= 0:
        display(error)
        print("Please input a solvent density greater than 0.")
        file_log.write("Please input a solvent density greater than 0.\n")
        return -1, -1, -1
    if rho_P <= 0:
        display(error)
        print("Please input a polymer density greater than 0.")
        file_log.write("Please input a polymer density greater than 0.\n")
        return -1, -1, -1

    # Check if the degree of polymerization <= 1
    if DP <= 1:
        display(error)
        print("Please input a degree of polymerization greater than 1.")
        print("If you wish to calculate the activity coefficient between small molecules, please use UNIFAC.")
        file_log.write("Please input a degree of polymerization greater than 1.\n")
        file_log.write("If you wish to calculate the activity coefficient between small molecules, please use UNIFAC.\n")
        return -1, -1, -1

    # check if fractions less than 1
    for i in range(0, len(frac)):
        if frac[i] >= 1:
            display(error)
            print("The fractions inputted must be less than 1.")
            print("Current fraction inputted: " + str(frac[i]))
            file_log.write("The fractions inputted must be less than 1.\n")
            file_log.write("Current fraction inputted: " + str(frac[i]) + "\n")
            return -1, -1, -1

    # Warnings (Not error)
    warnings = 0
    if rho_S > 1.5:
        print("!!!WARNING!!!")
        print("The density of the solvent, rho_S, appears to be quite high, make sure this is not a typo.")
        print("Note: The units of density is g/cm^3")
        warnings += 1
        file_log.write("!!!WARNING!!!\n")
        file_log.write("The density of the solvent, rho_S, appears to be quite high, make sure this is not a typo.\n")
        file_log.write("Note: The units of density is g/cm^3\n")

    if rho_P > 1.5:
        print("!!!WARNING!!!")
        print("The density of the polymer rho_P, appears to be quite high, make sure this is not a typo.")
        print("Note: The units of density is g/cm^3")
        warnings += 1
        file_log.write("!!!WARNING!!!\n")
        file_log.write("The density of the polymer rho_P, appears to be quite high, make sure this is not a typo.\n")
        file_log.write("Note: The units of density is g/cm^3\n")

    if rho_S < 0.5:
        print("!!!WARNING!!!")
        print("The density of the solvent, rho_S, appears to be quite low, make sure this is not a typo.")
        print("Note: The units of density is g/cm^3")
        warnings += 1
        file_log.write("!!!WARNING!!!\n")
        file_log.write("The density of the solvent, rho_S, appears to be quite low, make sure this is not a typo.\n")
        file_log.write("Note: The units of density is g/cm^3\n")

    if rho_P < 0.5:
        print("!!!WARNING!!!")
        print("The density of the polymer rho_P, appears to be quite low, make sure this is not a typo.")
        print("Note: The units of density is g/cm^3")
        warnings += 1
        file_log.write("!!!WARNING!!!\n")
        file_log.write("The density of the polymer rho_P, appears to be quite low, make sure this is not a typo.\n")
        file_log.write("Note: The units of density is g/cm^3\n")


    # Load in parameters
    wb = load_workbook(filename='Parameters.xlsx', read_only=True)
    ws1 = wb['Group Parameters']
    ws2 = wb['Interaction Parameters']

    df_group = pd.DataFrame(ws1.values)
    df_interaction = pd.DataFrame(ws2.values)

    wb.close()

    df_group.columns = df_group.iloc[0]
    df_group = df_group[1:]

    df_interaction.columns = df_interaction.iloc[0]
    df_interaction = df_interaction[1:]

    # print(df_group)
    # print(df_interaction)

    mainGroup = (df_group['Maingroup, m']).astype(str)
    mGroup = []
    for i in range(1, len(mainGroup)+1):
        m = mainGroup[i]
        a,b = str(m).split('[')
        c,d = str(b).split(']')
        mGroup.append(c)

    mGroup = np.array(mGroup)
    #print(mGroup)

    df_group['Maingroup, m'] = mGroup

    subGroup = (df_group['Subgroup Name']).astype(str)
    molecular_weight = []
    for i in range(1, len(subGroup)+1):
        molecular_weight.append(molar_mass(str(subGroup[i])))
    #print(molecular_weight)
    df_group['Molar Mass'] = molecular_weight

    #print(df_group)

    #########################
    for i in range(0, len(name_S)):
        #print(name_S[i])
        name_S[i] = (name_S[i]).strip()
        #print(name_S[i])
    for i in range(0, len(name_P)):
        name_P[i] = (name_P[i]).strip()

    # Check lengths
    if len(name_S) != len(vk_S):
        display(error)
        print("Error in solvent input.")
        print("The number of Subgroup Names should be equal to the number of subgroup amounts.")
        print("Currently have " + str(len(name_S)) + " names, and " + str(len(vk_S)) + " subgroup amounts.")
        print("Subgroup Names: " + str(name_S))
        print("Subgroup Amounts: " + str(vk_S))
        file_log.write("Error in solvent input.\n")
        file_log.write("The number of Subgroup Names should be equal to the number of subgroup amounts.\n")
        file_log.write("Currently have " + str(len(name_S)) + " names, and " + str(len(vk_S)) + " subgroup amounts.\n")
        file_log.write("Subgroup Names: " + str(name_S) + "\n")
        file_log.write("Subgroup Amounts: " + str(vk_S) + "\n")
        return -1, -1, -1

    if len(name_P) != len(vk_P):
        display(error)
        print("Error in polymer input.")
        print("The number of Subgroup Names should be equal to the number of subgroup amounts.")
        print("Currently have " + str(len(name_P)) + " names, and " + str(len(vk_P)) + " subgroup amounts.")
        print("Subgroup Names: " + str(name_P))
        print("Subgroup Amounts: " + str(vk_P))
        file_log.write("Error in polymer input.\n")
        file_log.write("The number of Subgroup Names should be equal to the number of subgroup amounts.\n")
        file_log.write("Currently have " + str(len(name_P)) + " names, and " + str(len(vk_P)) + " subgroup amounts.\n")
        file_log.write("Subgroup Names: " + str(name_P) + "\n")
        file_log.write("Subgroup Amounts: " + str(vk_P) + "\n")
        return -1, -1, -1


    group_S = [] # Subgroup number, k of each group in solvent
    M_SD = [] # Molar mass of each group in solvent
    Rk_S = [] # Rk value of each group in solvent
    Qk_S = [] # Qk value of each group in solvent

    for i in range(0, len(name_S)):
        df = df_group.loc[df_group['Subgroup Name'] == name_S[i]]
        if df.empty:
            display(error)
            print("Error in solvent input.")
            print("The Subgroup Name: " + str(name_S[i]) + " does not exist in our parameters")
            print("Please verify that you wrote the right subgroup name.")
            print("If you wish to add a new subgroup, please edit the parameters excel file.")
            file_log.write("Error in solvent input.\n")
            file_log.write("The Subgroup Name: " + str(name_S[i]) + " does not exist in our parameters.\n")
            file_log.write("Please verify that you wrote the right subgroup name.\n")
            file_log.write("If you wish to add a new subgroup, please edit the parameters excel file.\n")
            return -1, -1, -1
        group_S.append(int(df['Subgroup, k']))
        M_SD.append(float(df['Molar Mass']))
        Rk_S.append(float(df['R']))
        Qk_S.append(float(df['Q']))

    M_S = sum(np.multiply(vk_S, M_SD)) # Molar mass of solvent


    group_P = [] # Subgroup number, k of each group in polymer
    M_PD = [] # Molar mass of each group in polymer
    Rk_P = [] # Rk value of each group in polymer
    Qk_P = [] # Qk value of each group in polymer

    for i in range(0, len(name_P)):
        df = df_group.loc[df_group['Subgroup Name'] == name_P[i]]
        if df.empty:
            display(error)
            print("Error in polymer input.")
            print("The Subgroup Name: " + str(name_P[i]) + " does not exist in our parameters")
            print("Please verify that you wrote the right subgroup name.")
            print("If you wish to add a new subgroup, please edit the parameters excel file.")
            file_log.write("Error in polymer input.\n")
            file_log.write("The Subgroup Name: " + str(name_P[i]) + " does not exist in our parameters.\n")
            file_log.write("Please verify that you wrote the right subgroup name.\n")
            file_log.write("If you wish to add a new subgroup, please edit the parameters excel file.\n")
            return -1, -1, -1
        group_P.append(int(df['Subgroup, k']))
        M_PD.append(float(df['Molar Mass']))
        Rk_P.append(float(df['R']))
        Qk_P.append(float(df['Q']))

    M_P = sum(np.multiply(vk_P, M_PD)) # Molar mass of polymer

    #########################
    # interaction between solvent and polymer
    # Take in list of names output interaction matrix
    #print(name_S)
    names = np.concatenate((name_S, name_P))
    #print(names)
    #print(type(names))
    # For some reason the line below is not working
    names = np.unique(names)
    #print(names)
    #print(type(names))
    m = []
    for i in range(0, len(names)):
        df2 = df_group.loc[df_group['Subgroup Name'] == names[i]]
        #print(df2)
        m.append(int(df2['Maingroup, m']))
    m = np.sort(m)
    #print(m)
    a_SP = np.zeros((int(len(m)), int(len(m))))
    #print(a_SP)

    for i in range(0, len(m)):
        i_m = m[i]
        for j in range(0, len(m)):
            j_m = m[j]
            if not(i_m == j_m):
                if (i_m < j_m):
                    df3 = df_interaction[(df_interaction['i'] == i_m)]
                    df4 = df3[(df3['j'] == j_m)]
                    a_SP[i][j] = float(df4['Aij'])
                if (i_m > j_m):
                    df3 = df_interaction[(df_interaction['i'] == j_m)]
                    df4 = df3[(df3['j'] == i_m)]
                    a_SP[i][j] = float(df4['Aji'])
    #print(a_SP)


    ###########################################
    #w1 = [0.09575,0.05,0.02,0.01,0.001,0.00001] # vishnyakov value range
    a = np.zeros(len(frac))
    vol_frac2 = np.zeros(len(frac))
    a1_b1 = np.zeros(len(frac))
    a1_b = np.zeros(len(frac))

    for j in range(0, len(frac)):
        # only really need to print out one time to detect errors
        if (j==0):
            printed = True
        else:
            printed = False

        #(rho_S, rho_P, M1, M2, MS_d, MP_d, name_s, group_s, vk, Rk, Qk, name_p, group_p,
        #vk_p, Rk_p, Qk_p, a, w1, DP, Change, printed):
        # All values defined above, except change which changes UNIFAC to accept
        # volume fraction as input rather than w1

        a[j], vol_frac2[j], a1_b1[j], a1_b[j] = UNIFAC_FV(rho_S, rho_P, M_S, M_P, M_SD, M_PD, name_S, group_S, vk_S, Rk_S, Qk_S, name_P, group_P, vk_P, Rk_P, Qk_P, a_SP, frac[j], DP, T, frac_type, printed)
    print(a_SP)
    print()
    print("a (b = 1 or 1.28 depending on hard core volume)")
    print(a1_b)
    print()
    print("vol_frac2")
    print(vol_frac2)
    print()
    x = flory(vol_frac2, a1_b, DP)
    return warnings, x, a1_b


# REMINDER: COPY TO FUNCTIONS FILE (OR ELSE WON'T WORK)
def input_U(solvents, T, printed, file_log):

    # Polymers use different notations (very easy to get confused between them)
    # so for convenience I will make it it's own function
    # and not just copy stuff from UNIFAC-FV
    # Got this notation from Smith and Van ness book
    # Their way of writing the UNIFAC equations make it SIGNIFICANTLY easier
    # than Oishi's way of expressing the same thing

    # Initial Error testing
    warnings = 0
    # Check if Solvent # greater than 1
    if len(solvents) <= 1:
        display(error)
        print("Please input more than 1 solvent")
        file_log.write("Please input more than 1 solvent.\n")
        return -1, -1

    # Check if sum of all mole fraction == 1
    x_test = 0
    for i in range(0, len(solvents)):
        x_test += float(solvents[i][3])

    if np.round(x_test, 3) != 1:
        display(error)
        print("Sum of mole fractions must equal 1.")
        print("Currently it is: " + str(x_test))
        file_log.write("Sum of mole fractions must equal 1.\n")
        file_log.write("Currently it is: " + str(x_test) + "\n")
        return -1, -1

    # Check if T > 0
    if T <= 0:
        display(error)
        print("Please input a temperature greater than 0.")
        print("Note: The temperature unit is Kelvin.")
        file_log.write("Please input a temperature greater than 0.\n")
        file_log.write("Note: The temperature unit is Kelvin.\n")
        return -1, -1

    # Load in Parameters
    ###########################
    wb = load_workbook(filename='Parameters.xlsx', read_only=True)
    ws1 = wb['Group Parameters']
    ws2 = wb['Interaction Parameters']

    df_group = pd.DataFrame(ws1.values)
    df_interaction = pd.DataFrame(ws2.values)

    wb.close()

    df_group.columns = df_group.iloc[0]
    df_group = df_group[1:]

    df_interaction.columns = df_interaction.iloc[0]
    df_interaction = df_interaction[1:]

    # print(df_group)
    # print(df_interaction)

    mainGroup = (df_group['Maingroup, m']).astype(str)
    mGroup = []
    for i in range(1, len(mainGroup)+1):
        m = mainGroup[i]
        a,b = str(m).split('[')
        c,d = str(b).split(']')
        mGroup.append(c)

    mGroup = np.array(mGroup)
    #print(mGroup)

    df_group['Maingroup, m'] = mGroup

    subGroup = (df_group['Subgroup Name']).astype(str)
    molecular_weight = []
    for i in range(1, len(subGroup)+1):
        molecular_weight.append(molar_mass(str(subGroup[i])))
    #print(molecular_weight)
    df_group['Molar Mass'] = molecular_weight

    #print(df_group)

    #########################

    # For each group in each solvent define these parameters (From Excel file)
    # # k identifies subgroup values
    # i identifies species
    # so ik is species i, group k

    R_ik = [] # R_k subgroup parameter (based on Area)
    Q_ik = [] # Q_k subgroup parameter (based on Volume)
    ik = [] # k = subgroup number
    im = [] # m = maingroup number
    name_i = [] # names of each solvent
    subgroup_ik = [] #names of subgrops
    v_ik = [] # amount of subgroups
    MW_ik = [] # Molar weight
    x_i = [] # Mole fraction
    if printed:
        print("Before Populate Check")

    # Populate listed arrays above from parameter file
    for solvent in range(0, len(solvents)):
        if printed:
            print("Solvent: " + str(solvents[solvent][0]))
        name_i.append(solvents[solvent][0]) #subscript i identifies species
        subgroup_i = solvents[solvent][1] # subgroup names
        #print("subgroup_i = '" + str(subgroup_i) + "'")
        vk = solvents[solvent][2]
        v_ik.append(vk)
        x_i.append(solvents[solvent][3])

        if len(subgroup_i) == 0:
            display(error)
            print("The number of subgroup names has to be greater than 0.")
            print("Subgroup Names: " + str(subgroup_i))
            file_log.write("The number of subgroup names has to be greater than 0.\n")
            file_log.write("Subgroup Names: " + str(subgroup_i) + " \n")
            return -1, -1

        if len(vk) == 0:
            display(error)
            print("The number of subgroup amounts has to be greater than 0.")
            print("Subgroup Amounts: " + str(vk))
            file_log.write("The number of subgroup amounts has to be greater than 0.\n")
            file_log.write("Subgroup Amounts: " + str(vk) + "\n")
            return -1, -1

        if len(subgroup_i) != len(vk):
            display(error)
            print("The number of Subgroup Names should be equal to the number of subgroup amounts.")
            print("Currently have " + str(len(subgroup_i)) + " names, and " + str(len(vk)) + " subgroup amounts.")
            print("Subgroup Names: " + str(subgroup_i))
            print("Subgroup Amounts: " + str(vk))
            file_log.write("The number of Subgroup Names should be equal to the number of subgroup amounts. \n")
            file_log.write("Currently have " + str(len(subgroup_i)) + " names, and " + str(len(vk)) + " subgroup amounts. \n")
            file_log.write("Subgroup Names: " + str(subgroup_i) + "\n")
            file_log.write("Subgroup Amounts: " + str(vk) + "\n")
            return -1, -1


        R_k = []
        Q_k = []
        k = []
        m = []
        MW_k = []
        for subgroup in range(0, len(subgroup_i)):
            df = df_group.loc[df_group['Subgroup Name'] == (str(subgroup_i[subgroup])).strip()]
            if df.empty:
                display(error)
                print("The Subgroup Name: " + (str(subgroup_i[subgroup])).strip() + " does not exist in our parameters")
                print("Please verify that you wrote the right subgroup name.")
                print("If you wish to add a new subgroup, please edit the parameters excel file.")
                file_log.write("The Subgroup Name: " + (str(subgroup_i[subgroup])).strip() + " does not exist in our parameters. \n")
                file_log.write("Please verify that you wrote the right subgroup name.\n")
                file_log.write("If you wish to add a new subgroup, please edit the parameters excel file.\n")
                return -1, -1
            k.append(int(df['Subgroup, k']))
            m.append(int(df['Maingroup, m']))
            MW_k.append(float(df['Molar Mass']))
            R_k.append(float(df['R']))
            Q_k.append(float(df['Q']))

        subgroup_ik.append(subgroup_i)
        ik.append(k)
        im.append(m)
        R_ik.append(R_k)
        Q_ik.append(Q_k)
        MW_ik.append(MW_k)

    # Sort by order of increasing ik
    for solvent in range(0, len(solvents)):
        ik[solvent][:], im[solvent][:], R_ik[solvent][:], Q_ik[solvent][:], MW_ik[solvent][:] = zip(*sorted(zip(ik[solvent][:], im[solvent][:], R_ik[solvent][:], Q_ik[solvent][:], MW_ik[solvent][:])))

    if printed:
        print("Done Populating Arrays")
        # Turn lists to numpy arrays (Maybe not needed)
        print("ik")
        print(ik)
        print("im")
        print(im)
        print("v_ik")
        print(v_ik)
        print("R_ik")
        print(R_ik)
        print("Q_ik")
        print(Q_ik)

    r_i = []
    q_i = []
    for i in range(0, len(ik)):
        #print("i: " + str(i))
        r_i.append(0)
        q_i.append(0)
        for k in range(0, len(ik[i])):
            #print(ik[i])
            r_i[i] += v_ik[i][k]*R_ik[i][k]
            q_i[i] += v_ik[i][k]*Q_ik[i][k]
    if printed:
        print("r_i")
        print(r_i)
        print("q_i")
        print(q_i)

    e_ik = []
    for i in range(0, len(ik)):
        e_i = []
        for k in range(0, len(ik[i])):
            e_i.append((v_ik[i][k]*Q_ik[i][k])/(q_i[i]))
        e_ik.append(np.asarray(e_i))
#     print("e_ik")
#     print(e_ik)

    lengths = []
    #print(lengths)
    for i in range(0, len(ik)):
        lengths.append(int(len(ik[i])))
    lengths = np.asarray(lengths)
    max_k = np.amax(lengths)

    e_ki = np.zeros((max_k, len(ik)))
    for k in range(0, max_k):
        for i in range(0, len(ik)):
            try:
                e_ki[k][i] = e_ik[i][k]
            except:
                e_ki[k][i] = 0
    if printed:
        print("e_ki")
        print(e_ki)

    k_arr = []
    m_arr = []
    for i in range(0, len(ik)):
        for k in range(0, len(ik[i])):
            k_arr.append(ik[i][k])
            m_arr.append(im[i][k])
    k_arr = np.asarray(k_arr)
    m_arr = np.asarray(m_arr)
    dict_km = dict(zip(k_arr, m_arr))
    k_arr = np.unique(k_arr)
    m_arr = []
    for k in range(0, len(k_arr)):
        m_arr.append(dict_km[k_arr[k]])
    if printed:
        print("Subgroup, k")
        print(ik)
        print("Unique, k")
        print(k_arr)
        print("Corresponding, m")
        print(m_arr)


    a_ij = np.zeros((len(m_arr), len(m_arr)))
    for i in range(0, len(m_arr)):
        i_m = m_arr[i]
        for j in range(0, len(m_arr)):
            j_m = m_arr[j]
            if not(i_m == j_m):
                if (i_m < j_m):
                    df3 = df_interaction[(df_interaction['i'] == i_m)]
                    df4 = df3[(df3['j'] == j_m)]
                    a_ij[i][j] = float(df4['Aij'])
                if (i_m > j_m):
                    df3 = df_interaction[(df_interaction['i'] == j_m)]
                    df4 = df3[(df3['j'] == i_m)]
                    a_ij[i][j] = float(df4['Aji'])
    if printed:
        print("a_ij")
        print(a_ij)

    tau_ij = np.zeros((len(m_arr), len(m_arr)))

    for i in range(0, len(m_arr)):
        for j in range(0, len(m_arr)):
            tau_ij[i][j] = np.exp(-a_ij[i][j]/T)
    if printed:
        print("tau_ij")
        print(tau_ij)

    B_ik = []

    for i in range(0, len(ik)):
        B_k = []
        for k in range(0, max_k):
            B = []
            for m in range(0, len(ik[i])):
                B.append(e_ki[m][i]*tau_ij[m][k])
            B_k.append(np.sum(B))
        B_ik.append(B_k)
    if printed:
        print("B_ik")
        print(B_ik)

    theta = np.zeros(max_k)
    for k in range(0, max_k):
        theta_num = 0 # Numerator
        theta_den = 0 # Denominator
        for i in range(0, len(ik)):
            theta_num += x_i[i]*q_i[i]*e_ki[k][i]
        for j in range(0, len(ik)):
            theta_den += x_i[j]*q_i[j]
        theta[k] = theta_num/theta_den
    if printed:
        print("Theta_k")
        print(theta)

    s_k = np.zeros(max_k)
    for k in range(0, max_k):
        s_k[k] = 0
        for m in range(0, max_k):
            s_k[k] += theta[m]*tau_ij[m][k]
    if printed:
        print("S_k")
        print(s_k)

    J_i = np.zeros(len(ik))
    for i in range(0, len(ik)):
        J_den = 0
        for j in range(0, len(ik)):
            J_den += r_i[j]*x_i[j]
        J_i[i] = r_i[i]/J_den
    if printed:
        print("J_i")
        print(J_i)

    L_i = np.zeros(len(ik))
    for i in range(0, len(ik)):
        L_den = 0
        for j in range(0, len(ik)):
            L_den += q_i[j]*x_i[j]
        L_i[i] = q_i[i]/L_den
    if printed:
        print("L_i")
        print(L_i)

    # Y = activity coefficient
    ln_Y_Ci = np.zeros(len(ik)) # Combinatorial part
    ln_Y_Ri = np.zeros(len(ik)) # Residual part
    for i in range(0, len(ik)):
        ln_Y_Ci[i] = 1 - J_i[i] + np.log(J_i[i]) - 5*q_i[i]*(1 - (J_i[i]/L_i[i]) + np.log(J_i[i]/L_i[i]))
        r_sum = 0
        for k in range(0, max_k):
            r_sum += theta[k]*(B_ik[i][k]/s_k[k]) - e_ki[k][i]*np.log(B_ik[i][k]/s_k[k])
        ln_Y_Ri[i] = q_i[i]*(1 - r_sum)
    if printed:
        print("Combinatorial")
        print(ln_Y_Ci)
        print("Residual")
        print(ln_Y_Ri)

    ln_Y_i = np.zeros(len(ik))
    Y_i = np.zeros(len(ik))
    for i in range(0, len(ik)):
        ln_Y_i[i] = ln_Y_Ci[i] + ln_Y_Ri[i]
        Y_i[i] = np.exp(ln_Y_i[i])
        print("Activity coefficient for " + str(solvents[i][0]) + ": " + str(Y_i[i]))


    return warnings, Y_i

#

In [13]:
def read_input():

    wb = load_workbook(filename='Components.xlsx', read_only=True)
    ws1 = wb['Polymers']
    ws2 = wb['Solvents']

    df_p = pd.DataFrame(ws1.values)
    df_s = pd.DataFrame(ws2.values)

    wb.close()

    df_p.columns = df_p.iloc[0]
    df_p = df_p[1:]

    df_s.columns = df_s.iloc[0]
    df_s = df_s[1:]

    #



    # Perhaps more efficient to only read files you need rather than ask for them as input
    filenames = []
    filenames1 = glob.glob("*.UFVinp")
    filenames2 = glob.glob("*.Uinp")
    filenames = np.concatenate((filenames1, filenames2))
    print(filenames)
    for i in range(0, len(filenames)):
        file = filenames[i]
        base, extension = file.split('.')
        if extension.lower() == "ufvinp":
            log_name = str(base) + ".UFVlog"
            file_log = open(log_name, "w")
            with open(file) as f:
                line = f.readline()
                a, b = line.split(':')
                solvent_name = b.strip()
                if (a.strip()).lower() != "solvent":
                    display(error)
                    print("In file: " + str(file))
                    print("The first line of the .UFVinp file should be:  'Solvent: [solvent name]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The first line of the .UFVinp file should be:  'Solvent: [solvent name]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                if not solvent_name:
                    display(error)
                    print("In file: " + str(file))
                    print("The first line of the .UFVinp file should be:  'Solvent: [solvent name]'.")
                    print("The [solvent name] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The first line of the .UFVinp file should be:  'Solvent: [solvent name]'.\n")
                    file_log.write("The [solvent name] should not be empty. \n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                df = df_s.loc[df_s['Name'] == solvent_name]
                if df.empty:
                    display(error)
                    print("In file: " + str(file))
                    print("No solvent name: " + str(solvent_name))
                    print("If you wish to add a new solvent, please edit the Components.xlsx file.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("No solvent name: " + str(solvent_name) + "\n")
                    file_log.write("If you wish to add a new solvent, please edit the Components.xlsx file.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                #print(df['Subgroups'])
                name_S = (df['Subgroups']).to_numpy()
                name_S = str(name_S[0]).split(",")
                #print(name_S)
                vk_S = (df['Amounts']).to_numpy()
                #print(vk_S[0])
                vk_S = (np.array(str(vk_S[0]).split(","))).astype(int)
                line = f.readline()
                a, b = line.split(':')
                polymer_name = b.strip()
                if (a.strip()).lower() != "polymer":
                    display(error)
                    print("In file: " + str(file))
                    print("The second line of the .UFVinp file should be:  'Polymer: [polymer name]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The second line of the .UFVinp file should be:  'Polymer: [polymer name]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                if not polymer_name:
                    display(error)
                    print("In file: " + str(file))
                    print("The second line of the .UFVinp file should be:  'Polymer: [polymer name]'.")
                    print("The [polymer name] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The second line of the .UFVinp file should be:  'Polymer: [polymer name]'.\n")
                    file_log.write("The [polymer name] should not be empty.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                df = df_p.loc[df_p['Name'] == polymer_name]
                if df.empty:
                    display(error)
                    print("In file: " + str(file))
                    print("No polymer name: " + str(polymer_name))
                    print("If you wish to add a new polymer, please edit the Components.xlsx file.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("No polymer name: " + str(polymer_name) + "\n")
                    file_log.write("If you wish to add a new polymer, please edit the Components.xlsx file.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                name_P = (df['Subgroups']).to_numpy()
                name_P = str(name_P[0]).split(",")
                vk_P = (df['Amounts']).to_numpy()
                vk_P = (np.array(str(vk_P[0]).split(","))).astype(int)
                line = f.readline()
                a, b = line.split(':')
                if (a.strip()).lower() != "solvent density (g/cm3)":
                    display(error)
                    print("In file: " + str(file))
                    print("The third line of the .UFVinp file should be:  'Solvent Density (g/cm3): [Solvent Density]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The third line of the .UFVinp file should be:  'Solvent Density (g/cm3): [Solvent Density]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                rho_S = b.strip()
                if not rho_S:
                    display(error)
                    print("In file: " + str(file))
                    print("The third line of the .UFVinp file should be:  'Solvent Density (g/cm3): [Solvent Density]'.")
                    print("The [Solvent Density] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The third line of the .UFVinp file should be:  'Solvent Density (g/cm3): [Solvent Density]'.\n")
                    file_log.write("The [Solvent Density] should not be empty.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                line = f.readline()
                a, b = line.split(':')
                if (a.strip()).lower() != "polymer density (g/cm3)":
                    display(error)
                    print("In file: " + str(file))
                    print("The fourth line of the .UFVinp file should be:  'Polymer Density (g/cm3): [Polymer Density]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The fourth line of the .UFVinp file should be:  'Polymer Density (g/cm3): [Polymer Density]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                rho_P = b.strip()
                if not rho_P:
                    display(error)
                    print("In file: " + str(file))
                    print("The fourth line of the .UFVinp file should be:  'Polymer Density (g/cm3): [Polymer Density]'.")
                    print("The [Polymer Density] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The fourth line of the .UFVinp file should be:  'Polymer Density (g/cm3): [Polymer Density]'.\n")
                    file_log.write("The [Polymer Density] should not be empty.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                line = f.readline()
                a, b = line.split(':')
                if (a.strip()).lower() != "degree of polymerization":
                    display(error)
                    print("In file: " + str(file))
                    print("The fifth line of the .UFVinp file should be:  'Degree of Polymerization: [Degree of Polymerization]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The fifth line of the .UFVinp file should be:  'Degree of Polymerization: [Degree of Polymerization]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                DP = b.strip()
                if not DP:
                    display(error)
                    print("In file: " + str(file))
                    print("The fifth line of the .UFVinp file should be:  'Degree of Polymerization: [Degree of Polymerization]'.")
                    print("The [Degree of Polymerization] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The fifth line of the .UFVinp file should be:  'Degree of Polymerization: [Degree of Polymerization]'.\n")
                    file_log.write("The [Degree of Polymerization] should not be empty.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                line = f.readline()
                a, b = line.split(':')
                #print(a)
                #print(a.strip())
                if (a.strip()).lower() != "temperature (k)":
                    display(error)
                    print("In file: " + str(file))
                    print("The sixth line of the .UFVinp file should be: 'Temperature (K): [Temperature]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The sixth line of the .UFVinp file should be: 'Temperature (K): [Temperature]'. \n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                T = b.strip()
                if not T:
                    display(error)
                    print("In file: " + str(file))
                    print("The sixth line of the .UFVinp file should be:  'Temperature (K): [Temperature]'.")
                    print("The [Temperature] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The sixth line of the .UFVinp file should be: 'Temperature (K): [Temperature]'. \n")
                    file_log.write("The [Temperature] should not be empty. \n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                line = f.readline()
                a, b = line.split(':')
                if (a.strip()).lower() != "type":
                    display(error)
                    print("In file: " + str(file))
                    print("The seventh line of the .UFVinp file should be:  'Type: [Type]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The seventh line of the .UFVinp file should be:  'Type: [Type]'. \n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                fraction_type = b.strip()
                f_type = (b.strip()).lower()
                if not f_type:
                    display(error)
                    print("In file: " + str(file))
                    print("The seventh line of the .UFVinp file should be:  'Type: [Type]'.")
                    print("The [Type] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The seventh line of the .UFVinp file should be:  'Type: [Type]'. \n")
                    file_log.write("The [Type] should not be empty. \n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                if (f_type == 'solvent weight') or (f_type == 'solvent weight fraction'):
                    frac_type = 1
                elif (f_type == 'solvent volume') or (f_type == 'solvent volume fraction'):
                    frac_type = 2
                elif (f_type == 'polymer weight fraction') or (f_type == 'polymer weight'):
                    frac_type = 3
                elif (f_type == 'polymer volume') or (f_type == 'polymer volume fraction'):
                    frac_type = 4
                else:
                    display(error)
                    print("In file: " + str(file))
                    print("Fraction type " + str(b) + " is not valid.")
                    print("Acceptable fraction types include: ")
                    print("Solvent Weight Fraction")
                    print("Solvent Volume Fraction")
                    print("Polymer Weight Fraction")
                    print("Polymer Volume Fraction")
                    print("Note: The 'fraction' at the end is optional.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("Fraction type " + str(b) + " is not valid.\n")
                    file_log.write("Acceptable fraction types include: \n")
                    file_log.write("Solvent Weight Fraction\n")
                    file_log.write("Solvent Volume Fraction\n")
                    file_log.write("Polymer Weight Fraction\n")
                    file_log.write("Polymer Volume Fraction\n")
                    file_log.write("Note: The 'fraction' at the end is optional.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                line = f.readline()
                a, b = line.split(':')
                if (a.strip()).lower() != "fraction":
                    display(error)
                    print("In file: " + str(file))
                    print("The eigth line of the .UFVinp file should be:  'Fraction: [Comma separated fraction list]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The eigth line of the .UFVinp file should be:  'Fraction: [Comma separated fraction list]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                frac = b.strip()
                if not frac:
                    display(error)
                    print("In file: " + str(file))
                    print("The eigth line of the .UFVinp file should be:  'Fraction: [Comma separated fraction list]'.")
                    print("The [Comma separated fraction list] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The eigth line of the .UFVinp file should be:  'Fraction: [Comma separated fraction list]'.\n")
                    file_log.write("The [Comma separated fraction list] should not be empty. \n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                frac = (np.array(str(frac).split(","))).astype(float)
                line = f.readline()
                if line:
                    if line != "\n":
                        display(error)
                        print("In file: " + str(file))
                        print("There should not be a ninth line in the .UFVinp file.")
                        print("Line: ")
                        print(line)
                        file_log.write("In file: " + str(file) + "\n")
                        file_log.write("There should not be a ninth line in the .UFVinp file.\n")
                        file_log.write("Line: \n")
                        file_log.write( str(line) + "\n")
                        file_log.write("Exited with errors.")
                        file_log.close()
                        return
            warnings, x, a1_b = input_UFV(name_S, name_P, vk_S, vk_P, float(rho_S), float(rho_P), float(DP), float(T), frac, frac_type, file_log)

            if int(warnings) == -1:
                file_log.write("Exited with errors.")
                return
            else:
                # print out info here
                file_log.write("Polymer Name: " + str(polymer_name) + "\n")
                file_log.write("Polymer Density (g/cm^3): " + str(rho_P) +"\n")
                file_log.write("Solvent Name: " + str(solvent_name) + "\n")
                file_log.write("Solvent Density (g/cm^3): " + str(rho_S) +"\n")
                file_log.write("Degree of Polymerization: " + str(DP) + "\n")
                file_log.write("Temperature (K): " + str(T) + "\n")
                file_log.write(str(fraction_type) + ": " + str(frac) + "\n")
                file_log.write("Activities: " + str(a1_b) + "\n")
                file_log.write("Flory-Huggins Parameter: " + str(np.round(x,3)) + "\n")
                if int(warnings) == 0:
                    # No errors or warnings
                    file_log.write("Exited with no errors or warnings.")
                elif int(warnings) == 1:
                    file_log.write("Exited with 1 warning.")
                elif int(warnings) >1:
                    file_log.write("Exited with " + str(int(warnings)) + " warnings.")
                else:
                    file_log.write("Warnings not listed in possible options.\n")
                    file_log.write("Exited with errors.")
            file_log.close()

        elif extension.lower() == "uinp":
            log_name = str(base) + ".Ulog"
            file_log = open(log_name, "w")
            with open(file) as f:
                line = f.readline()
                line_count = 1
                solvents = []
                a, b = line.split(':')
                mixture_number = b.strip()
                if (a.strip()).lower() != "mixture":
                    display(error)
                    print("In file: " + str(file))
                    print("The line " + str(line_count) +" of the .Uinp file should be:  'Mixture: [number of components]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Mixture: [number of components]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                if not mixture_number:
                    display(error)
                    print("In file: " + str(file))
                    print("The line " + str(line_count) +" of the .Uinp file should be:  'Mixture: [number of components]'.")
                    print("The [number of components] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Mixture: [number of components]'.\n")
                    file_log.write("The [number of components] should not be empty.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                mixture_number = int(mixture_number)
                for i in range(0, mixture_number*2):
                    # names = 0, 2, 4, etc... /2 + 1 = 1, 2, 3
                    # mole fraction = 1, 3, 5, etc...
                    line = f.readline()
                    line_count += 1
                    a, b = line.split(':')
                    if i % 2 == 0:
                        solvent_i = []
                        solvent_name = b.strip()
                        solvent_i.append(solvent_name)
                        comp_name = "component " + str(int((i/2) + 1))
                        if (a.strip()).lower() != comp_name:
                            display(error)
                            print("In file: " + str(file))
                            print("The line " + str(line_count) +" of the .Uinp file should be:  'Component " + str(int((i/2) + 1)) +": [Component Name]'.")
                            file_log.write("In file: " + str(file) + "\n")
                            file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Component " + str(int((i/2) + 1)) +": [Component Name]'.\n")
                            file_log.write("Exited with errors.")
                            file_log.close()
                            return
                        if not solvent_name:
                            display(error)
                            print("In file: " + str(file))
                            print("The line " + str(line_count) +" of the .Uinp file should be:  'Component " + str(int((i/2) + 1)) +": [Component Name]'.")
                            print("The [Component Name] should not be empty")
                            file_log.write("In file: " + str(file) + "\n")
                            file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Component " + str(int((i/2) + 1)) +": [Component Name]'.\n")
                            file_log.write("The [Component Name] should not be empty.\n")
                            file_log.write("Exited with errors.")
                            file_log.close()
                            return
                        df = df_s.loc[df_s['Name'] == solvent_name]
                        if df.empty:
                            display(error)
                            print("In file: " + str(file))
                            print("No solvent name: " + str(solvent_name))
                            print("If you wish to add a new solvent, please edit the Components.xlsx file.")
                            file_log.write("In file: " + str(file) + "\n")
                            file_log.write("No solvent name: " + str(solvent_name) + "\n")
                            file_log.write("If you wish to add a new solvent, please edit the Components.xlsx file.\n")
                            file_log.write("Exited with errors.")
                            file_log.close()
                            return
                        #print(df['Subgroups'])
                        name_S = (df['Subgroups']).to_numpy()
                        name_S = str(name_S[0]).split(",")
                        #print(name_S)
                        solvent_i.append(name_S)
                        vk_S = (df['Amounts']).to_numpy()
                        #print(vk_S[0])
                        vk_S = (np.array(str(vk_S[0]).split(","))).astype(int)
                        solvent_i.append(vk_S)
                    if i % 2 == 1:
                        mol_frac = b.strip()
                        mol_name = "mole fraction " + str(int(math.ceil(i/2)))
                        if (a.strip()).lower() != mol_name:
                            display(error)
                            print("In file: " + str(file))
                            print("The line " + str(line_count) +" of the .Uinp file should be:  'Mole Fraction " + str(int(math.ceil(i/2))) +": [Mole Fraction]'.")
                            file_log.write("In file: " + str(file) + "\n")
                            file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Mole Fraction " + str(int(math.ceil(i/2))) +": [Mole Fraction]'.\n")
                            file_log.write("Exited with errors.")
                            file_log.close()
                            return
                        if not mol_frac:
                            display(error)
                            print("In file: " + str(file))
                            print("The line " + str(line_count) +" of the .Uinp file should be:  'Mole Fraction " + str(int(math.ceil(i/2))) +": [Mole Fraction]'.")
                            print("The [Mole Fraction] should not be empty")
                            file_log.write("In file: " + str(file) + "\n")
                            file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Mole Fraction " + str(int(math.ceil(i/2))) +": [Mole Fraction]'.\n")
                            file_log.write("The [Mole Fraction] should not be empty.\n")
                            file_log.write("Exited with errors.")
                            file_log.close()
                            return
                        mol_frac = str(mol_frac).split(",")
                        mol_frac = (np.array(mol_frac)).astype(float)
                        solvent_i.append(mol_frac)
                        solvents.append(solvent_i)
                line = f.readline()
                line_count += 1
                a, b = line.split(':')
                T = b.strip()
                if (a.strip()).lower() != "temperature (k)":
                    display(error)
                    print("In file: " + str(file))
                    print("The line " + str(line_count) +" of the .Uinp file should be:  'Temperature (K): [Temperature]'.")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Temperature (K): [Temperature]'.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                if not T:
                    display(error)
                    print("In file: " + str(file))
                    print("The line " + str(line_count) +" of the .Uinp file should be:  'Temperature (K): [Temperature]'.")
                    print("The [Temperature] should not be empty")
                    file_log.write("In file: " + str(file) + "\n")
                    file_log.write("The line " + str(line_count) +" of the .Uinp file should be:  'Temperature (K): [Temperature]'.\n")
                    file_log.write("The [Temperature] should not be empty.\n")
                    file_log.write("Exited with errors.")
                    file_log.close()
                    return
                T = float(T)
                line = f.readline()
                if line:
                    if line != "\n":
                        display(error)
                        print("In file: " + str(file))
                        print("There should not be a ninth line in the .UFVinp file.")
                        print("Line: ")
                        print(line)
                        file_log.write("In file: " + str(file) + "\n")
                        file_log.write("There should not be a ninth line in the .UFVinp file.\n")
                        file_log.write("Line: \n")
                        file_log.write(str(line) + "\n")
                        file_log.write("Exited with errors.")
                        file_log.close()
                        return
            #print(solvents)
            printed = False

            # print("Mol_frac")
            # print(mol_frac)
            # print(len(mol_frac))

            activity_coef = []
            if len(mol_frac) > 1:
                coefs = np.zeros((len(solvents), len(solvents[0][3])))
                mol_fractions = np.zeros((len(solvents), len(solvents[0][3])))
                for i in range(0, len(solvents)):
                    mol_fractions[i][:] = solvents[i][3]

                for i in range(0, len(mol_fractions[0][:])):
                    for j in range(0, len(solvents)):
                        solvents[j][3] = mol_fractions[j][i]
                    print("For j: " + str(i))
                    print(solvents)
                    warnings, coefficients =  input_U(solvents, T, printed, file_log)
                    #print(coefficients)
                    if int(warnings) == -1:
                        file_log.write("Exited with errors.")
                        return
                    elif int(warnings) != -1:
                        for j in range(0, len(solvents)):
                            coefs[j][i] = coefficients[j]
                for i in range(0, len(solvents)):
                    activity_coef.append(coefs[i][:])


            elif len(mol_frac) == 1:
                warnings, coefficients =  input_U(solvents, T, printed)
                if warnings != -1:
                    for i in range(0, len(solvents)):
                        activity_coef.append(coefficients[i])


            if int(warnings) == -1:
                file_log.write("Exited with errors.")
                return
            else:
                # print out info here
                for i in range(0, len(solvents)):
                    file_log.write("For " + str(solvents[i][0]) + ": \n")
                    file_log.write("Mole Fraction: " + str(mol_fractions[i]) + "\n")
                    file_log.write("Activity Coefficient: " + str(np.round(activity_coef[i],3)) + "\n")
                    file_log.write("\n")
                if int(warnings) == 0:
                    # No errors or warnings
                    file_log.write("Exited with no errors or warnings.")
                elif int(warnings) == 1:
                    file_log.write("Exited with 1 warning.")
                elif int(warnings) >1:
                    file_log.write("Exited with " + str(int(warnings)) + " warnings.")
                else:
                    file_log.write("Warnings not listed in possible options.\n")
                    file_log.write("Exited with errors.")
            file_log.close()

        else: # Should never happen given the glob reading, but I might change glob reading to something else in the future
            display(error)
            print("File extension '" + str(file) + "' is not supported.")
            print("Please use file extensions '.UFVinp' for UNIFAC-FV or .Uinp for UNIFAC.")
            log_name = str(base) + ".ERRlog"
            file_log = open(log_name, "w")
            file_log.write("File extension '" + str(file) + "' is not supported.\n")
            file_log.write("Please use file extensions '.UFVinp' for UNIFAC-FV or .Uinp for UNIFAC.\n")
            file_log.write("Exited with errors.")
            file_log.close()
            return

    return

In [ ]:
# # Testing Area
# C = []
# A = [1, 2]
# B = [3, 4]
# C.append(A)
# C.append(B)
# print(C[0])
# print(C[1])
# print(C[0][0])
# C = np.asarray(C)
# print(C.flatten())
read_input()

['Example.UFVinp' 'Example.Uinp']
      k      Rk     Qk  V_k^(1)  V_k^(2)
CH3   1  0.9011  0.848        0        1
CH2   2  0.6744  0.540        0        1
CH    3  0.4469  0.228        0        1
CF3  74  1.4060  1.380        2        0
CF2  75  1.0105  0.920        6        0

[[  0.      0.      0.     -2.859  -2.859]
 [  0.      0.      0.     -2.859  -2.859]
 [  0.      0.      0.     -2.859  -2.859]
 [147.3   147.3   147.3     0.      0.   ]
 [147.3   147.3   147.3     0.      0.   ]]

a (b = 1 or 1.28 depending on hard core volume)
[5.36982678e+00 3.67807776e+00 1.77824323e+00 9.49365070e-01
 1.00818904e-01 1.01494607e-03]

vol_frac2
[0.90734935 0.95169757 0.98069972 0.9903533  0.99903564 0.99999036]

_________________________________________________
Flory-Huggins Parameter: 3.799755109327962
_________________________________________________
For j: 0
[['Diethylamine', ['CH3', ' CH2NH', ' CH2'], array([2, 1, 1]), 0.4], ['Heptane', ['CH3', ' CH2'], array([2, 5]), 0.6]]
